In [6]:
import sys        
import random
import warnings
import time
import numpy as np
import pandas as pd
import collections, functools, operator
import os
import pickle

# Parameters of system
warnings.filterwarnings("ignore")

In [7]:
# Precios iguales
#Escenario 1: Optimizo 21 - Implemento 21
#Escenario 2: Optimizo 21 - Implemento 14
#Escenario 3: Optimizo 21 - Implemento 7

# Precios libres
#Escenario 4: Optimizo 21 - Implemento 21
#Escenario 5: Optimizo 21 - Implemento 14
#Escenario 6: Optimizo 21 - Implemento 7

os.chdir('/Users/moises/Desktop/Produccion-Tesis/')

# Correr 30 replicas
replicas = 50
q = 15000

for experiment in [1,2,3,4,5,6]:            
    file_name = f"Resultados/{q}/Escenario {experiment}/"
    df = pd.read_csv(file_name+"objective_value.csv", sep=';')
    print(f"Escenario {experiment}: {df['value'].sum()/replicas} +- {df['value'].std()*1.96/np.sqrt(replicas)}")

Escenario 1: 217403430.4864659 +- 22661.018173933306
Escenario 2: 217338427.92543802 +- 20341.73635071743
Escenario 3: 217047503.26024327 +- 18200.55781342851
Escenario 4: 219068269.3226572 +- 31903.48777372496
Escenario 5: 219140469.66737053 +- 31582.10463283589
Escenario 6: 219054443.95510158 +- 31134.983239498684


In [8]:

#file_name = f"Resultados/Escenario 1/"
#df1 = pd.read_csv(file_name+"objective_value.xlsx", sep=";")
#
#file_name = f"Resultados/Escenario 2/"
#df2 = pd.read_csv(file_name+"objective_value.xlsx")
#
#df = df1.merge(df2, on=['t','r'])
#df['dif'] = df.value_x - df.value_y
#print(f"{df.groupby('r').sum()['dif'].mean()} + {df.groupby('r').sum()['dif'].std()*1.96/np.sqrt(10)}")

In [16]:
from scipy import stats

final = pd.DataFrame()
for experiment in range (1, 7):
    # Recorrer cada experimento
    if experiment in [1, 2,  3, 4, 5, 6]:
        file_name = f"Resultados/{q}/Escenario {experiment}/"
        df = pd.read_csv(file_name+"objective_value.csv", sep=";").rename(columns={'value': 'utilidad'}).groupby(['r']).sum().reset_index()['utilidad']
        df1 = pd.read_csv(file_name+"ingresos.csv", sep=";").rename(columns={'value': 'ingresos'}).groupby(['r']).sum().reset_index()['ingresos']
        df2 = pd.read_csv(file_name+"costo_inventario.csv", sep=";").rename(columns={'value': 'costo_inventario'}).groupby(['r']).sum().reset_index()['costo_inventario']
        df3 = pd.read_csv(file_name+"costo_corte.csv", sep=";").rename(columns={'value': 'costo_corte'}).groupby(['r']).sum().reset_index()['costo_corte']
        df4 = pd.read_csv(file_name+"costo_merma.csv", sep=";").rename(columns={'value': 'costo_merma'}).groupby(['r']).sum().reset_index()['costo_merma']
        df = pd.concat([df, df1, df2, df3, df4], axis=1)
        df['Escenario'] = f"Escenario {experiment}"
        df = df.reset_index().rename(columns={'index': 'r'})
        df['r'] += 1

        # Se agrega al DataFrame 
        final = pd.concat([final, df])
        
        fila = f"Escenario {experiment} & "
        for metrica in ['utilidad', 'ingresos', 'costo_inventario', 'costo_corte', 'costo_merma']:
            # Calcula la media y desviación estándar de la columna 'utilidad'
            media = df[metrica].mean()
            desviacion_estandar = df[metrica].std()

            # Calcula el tamaño de la muestra
            tamano_muestra = len(df)
            
            # Define el nivel de confianza deseado (por ejemplo, 95%)
            nivel_confianza = 0.95

            # Calcula el valor crítico de la distribución t de Student
            valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, tamano_muestra - 1)

            # Calcula el error estándar
            error_estandar = desviacion_estandar / np.sqrt(tamano_muestra)

            # Calcula el intervalo de confianza
            intervalo_inferior = media - valor_critico * error_estandar
            intervalo_superior = media + valor_critico * error_estandar

            # Expresa el intervalo de confianza en potencia de 10e6 (potencia de 6)
            media = "{:.3f}".format(media * 1e-6)
            error = "{:.3f}".format(valor_critico * error_estandar * 1e-6)

            # Imprime el intervalo de confianza expresado en potencia de 10e6 (potencia de 6)
            cadena = f"${media}$ $\pm$ ${error}$ $\\times10^-6_$".replace('-', '{'). replace('_', '}')
            #print(f"{metrica}:{cadena}")

            fila += f"{cadena} & "

        fila =  fila[:-3]
        if experiment != 6:
            fila += "\\\\ \\midrule "
        else:
            fila += "\\\\ \\bottomrule "

        print(fila)

        
#final
#final.to_excel("/Users/moises/Desktop/Codigo - Test significancia/ANOVA.xlsx", sheet_name="ANOVA")    


Escenario 1 & $215.390$ $\pm$ $0.129$ $\times10^{6}$ & $248.230$ $\pm$ $0.113$ $\times10^{6}$ & $2.253$ $\pm$ $0.030$ $\times10^{6}$ & $30.475$ $\pm$ $0.009$ $\times10^{6}$ & $0.112$ $\pm$ $0.001$ $\times10^{6}$\\ \midrule 
Escenario 2 & $215.406$ $\pm$ $0.129$ $\times10^{6}$ & $248.439$ $\pm$ $0.116$ $\times10^{6}$ & $2.465$ $\pm$ $0.027$ $\times10^{6}$ & $30.456$ $\pm$ $0.010$ $\times10^{6}$ & $0.112$ $\pm$ $0.001$ $\times10^{6}$\\ \midrule 
Escenario 3 & $215.216$ $\pm$ $0.143$ $\times10^{6}$ & $248.554$ $\pm$ $0.124$ $\times10^{6}$ & $2.669$ $\pm$ $0.030$ $\times10^{6}$ & $30.554$ $\pm$ $0.015$ $\times10^{6}$ & $0.115$ $\pm$ $0.001$ $\times10^{6}$\\ \midrule 
Escenario 4 & $216.903$ $\pm$ $0.126$ $\times10^{6}$ & $249.246$ $\pm$ $0.110$ $\times10^{6}$ & $1.821$ $\pm$ $0.025$ $\times10^{6}$ & $30.474$ $\pm$ $0.009$ $\times10^{6}$ & $0.048$ $\pm$ $0.002$ $\times10^{6}$\\ \midrule 
Escenario 5 & $216.972$ $\pm$ $0.119$ $\times10^{6}$ & $249.181$ $\pm$ $0.108$ $\times10^{6}$ & $1.667$ 